In [1]:
"""
Created on Fri Feb 19 08:46:16 2021

@author: Schroeder
"""

# Liest holc etc. daten von bereits auf platte befindlichen csv dataframes .
# Soll Plot und Auswertung von datagrabbing trennen.

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display

#import TickerSelector, regression
from datetime import datetime
from datetime import date
import calendar

# Imports
from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import scipy.stats as stats


from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
import csv
   
#import StockscreenerPlotter_stats
#import StockscreenerWinners_stats
#import downloader

import sys
import os.path
from os.path import isfile, join

from os import listdir
#from os.path import isfile, join

#currentdir = os.path.join(os.path.dirname(__file__))

###### Insert the following three lines to make any import lib in he project dir setup visible to an other
###### Directory in the project setup

currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

from lib import Indikatoren

#lieferrt bei gegebenem datatfram mit datum als index das erste Datum, das letzte, und zwei 
# dazwischen


    

## SCANNER sucht in Data Universum (etwa SP500 oder Aristokraten) nach Stocks, die ein bestimmtes Signal aufweisen. Also etwa 
## weekly MACD positiv und RSI > 75% etc.

## Gebe Pfad an, unter dem das Datenfile zu finden ist.
## Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/


# --------------------------------------------------------------------------
## Welcher Sektor oder index soll analysieert werden ?

In [2]:
pfad = os.path.realpath(os.path.join(parentdir, 'ETFS'))

#############################################################
who = "RES_SP500"
pfad = os.path.realpath(os.path.join(pfad, who))
###############################################################
pfad = os.path.realpath(os.path.join(pfad, 'Data'))


file  = "_ticker_names.csv"

singleticker_file=os.path.realpath(os.path.join(pfad, file))



In [3]:
exportList = pd.DataFrame(columns=['ticker',"z-score","Volume","MACDsignal","RSIlevel","Name","Industry","Sector","marketCap"])

## ---------------------------------------------------------------------

In [4]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    mydict={}
    file=pfad+ "/"  + i + ".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    #########################################################################
    if (df["z-score"][-1] < -2.5) or (df["z-score"][-1] >2.5):
    ########################################################################
        mydict["ticker"]=i
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
        mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Short"        
    
    
    if i == "HRB":
        fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
        fig.show()  

    if len(mydict)>0:
        exportList =exportList.append(mydict,ignore_index=True)  



In [ ]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    mydict={}
    file=pfad+ "/"  + i + ".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    #########################################################################
    if (df["z-score"][-1] < -2.5) or (df["z-score"][-1] >2.5):
    ########################################################################
        mydict["ticker"]=i
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
        mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Short"        
    
    
    if i == "HRB":
        fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
        fig.show()  

    if len(mydict)>0:
        exportList =exportList.append(mydict,ignore_index=True)  



In [5]:
exportList = exportList.sort_values(by="z-score",ascending=False)
exportList=exportList.fillna("-")
pd.set_option("display.max_rows",None)
display(exportList)

,ticker,z-score,Volume,MACDsignal,RSIlevel,Name,Industry,Sector,marketCap
158,SGFY,3.095474,-,-,-,"1754 Signify Health, Inc. Name: name, dtype...",1754 Health Information Services Name: indu...,"1754 Healthcare Name: sector, dtype: object",-
8,AGS,2.953882,-,-,-,"939 PlayAGS, Inc. Name: name, dtype: object","939 Gambling Name: industry, dtype: object","939 Consumer Cyclical Name: sector, dtype: ...",-
85,HIL,2.933244,237410,Long,-,"649 Hill International, Inc. Name: name, dt...",649 Engineering & Construction Name: indust...,"649 Industrials Name: sector, dtype: object",-
133,OSG,2.795616,-,-,-,"897 Overseas Shipholding Group, Inc. Name: ...","897 Oil & Gas Midstream Name: industry, dty...","897 Energy Name: sector, dtype: object",-
64,FL,2.706837,-,-,-,"562 Foot Locker, Inc. Name: name, dtype: ob...","562 Apparel Retail Name: industry, dtype: o...","562 Consumer Cyclical Name: sector, dtype: ...",-
47,DEN,2.650658,527546,Long,-,"1700 Denbury Inc. Name: name, dtype: object","1700 Oil & Gas E&P Name: industry, dtype: o...","1700 Energy Name: sector, dtype: object",-
206,WEBR,2.631287,-,-,-,"1784 Weber Inc. Name: name, dtype: object","1784 Furnishings, Fixtures & Appliances Nam...","1784 Consumer Cyclical Name: sector, dtype:...",-
16,APRN,2.598334,-,-,-,"1570 Blue Apron Holdings, Inc. Name: name, ...","1570 Internet Retail Name: industry, dtype:...","1570 Consumer Cyclical Name: sector, dtype:...",-
72,GHM,2.532912,-,-,-,"602 Graham Corporation Name: name, dtype: o...",602 Specialty Industrial Machinery Name: in...,"602 Industrials Name: sector, dtype: object",-
200,VPG,2.511371,35403,Long,-,"1195 Vishay Precision Group, Inc. Name: nam...",1195 Scientific & Technical Instruments Nam...,"1195 Technology Name: sector, dtype: object",-
